In [2]:
import csv
import os
import numpy as np

def load_csv_data_with_dictreader(data_path, sub_sample=False):
    '''
    Since csv load function in helpers.py is too slow, we implemented faster reader. 
    load_csv_data function need more than 1 hour on EPFL noto server.
    Still the fastest choice is pandas dataframe. I hope you allow pandas for proj 1 next year.
    '''
    def csv_to_np_arrays(file_path):
        with open(file_path, 'r') as f:
            reader = csv.DictReader(f)
            headers = reader.fieldnames            
            data = [list(map(lambda x: float(x) if x else 0.0, row.values())) for row in reader]            
            return np.array(data), headers
        
    y_train_data, _ = csv_to_np_arrays(os.path.join(data_path, "y_train.csv"))
    x_test_data, _ = csv_to_np_arrays(os.path.join(data_path, "x_test.csv"))
    x_train_data, x_headers = csv_to_np_arrays(os.path.join(data_path, "x_train.csv"))
    y_train = y_train_data[:, 1].astype(dtype=int)
    x_train = x_train_data[:, 1:]
    x_test = x_test_data[:, 1:]
    train_ids = x_train_data[:, 0].astype(dtype=int)
    test_ids = x_test_data[:, 0].astype(dtype=int)

    # sub-sample
    if sub_sample:
        y_train = y_train[::50]
        x_train = x_train[::50]
        train_ids = train_ids[::50]
    return x_train, x_test, y_train, train_ids, test_ids, x_headers

In [3]:
x_train, x_test, y_train, train_ids, test_ids, x_headers = load_csv_data_with_dictreader('data')

small done


In [6]:
x_headers

['Id',
 '_STATE',
 'FMONTH',
 'IDATE',
 'IMONTH',
 'IDAY',
 'IYEAR',
 'DISPCODE',
 'SEQNO',
 '_PSU',
 'CTELENUM',
 'PVTRESD1',
 'COLGHOUS',
 'STATERES',
 'CELLFON3',
 'LADULT',
 'NUMADULT',
 'NUMMEN',
 'NUMWOMEN',
 'CTELNUM1',
 'CELLFON2',
 'CADULT',
 'PVTRESD2',
 'CCLGHOUS',
 'CSTATE',
 'LANDLINE',
 'HHADULT',
 'GENHLTH',
 'PHYSHLTH',
 'MENTHLTH',
 'POORHLTH',
 'HLTHPLN1',
 'PERSDOC2',
 'MEDCOST',
 'CHECKUP1',
 'BPHIGH4',
 'BPMEDS',
 'BLOODCHO',
 'CHOLCHK',
 'TOLDHI2',
 'CVDSTRK3',
 'ASTHMA3',
 'ASTHNOW',
 'CHCSCNCR',
 'CHCOCNCR',
 'CHCCOPD1',
 'HAVARTH3',
 'ADDEPEV2',
 'CHCKIDNY',
 'DIABETE3',
 'DIABAGE2',
 'SEX',
 'MARITAL',
 'EDUCA',
 'RENTHOM1',
 'NUMHHOL2',
 'NUMPHON2',
 'CPDEMO1',
 'VETERAN3',
 'EMPLOY1',
 'CHILDREN',
 'INCOME2',
 'INTERNET',
 'WEIGHT2',
 'HEIGHT3',
 'PREGNANT',
 'QLACTLM2',
 'USEEQUIP',
 'BLIND',
 'DECIDE',
 'DIFFWALK',
 'DIFFDRES',
 'DIFFALON',
 'SMOKE100',
 'SMOKDAY2',
 'STOPSMK2',
 'LASTSMK2',
 'USENOW3',
 'ALCDAY5',
 'AVEDRNK2',
 'DRNK3GE5',
 'MAXDRNKS',
 '

In [7]:
x_train.shape

(328135, 321)

### preprocess data

In [333]:
def min_max_scale(data):
    min_vals = np.min(data, axis=0)
    max_vals = np.max(data, axis=0)
    denominators = max_vals - min_vals + 1e-10
    scaled_data = (data - min_vals) / denominators
    
    return scaled_data

In [357]:
scaled_x_train = min_max_scale(x_train)
scaled_x_test = min_max_scale(x_test)

In [358]:
for i in range(x_train.shape[1]):
    print(i,x_headers[i+1],'\t',np.mean(scaled_x_train[:,i]),'\t',np.std(scaled_x_train[:,i]))

0 _STATE 	 0.4080795927984214 	 0.22579933626514148
1 FMONTH 	 0.4873354508978452 	 0.317023382543091
2 IDATE 	 0.4912940433133758 	 0.30878253630892033
3 IMONTH 	 0.49242371141141794 	 0.317521242734204
4 IDAY 	 0.4498744927123734 	 0.2778160783974894
5 IYEAR 	 0.024758102607730356 	 0.15538706175393
6 DISPCODE 	 0.15014856690066813 	 0.35721698553987263
7 SEQNO 	 0.22365188130326458 	 0.17726044212959832
8 _PSU 	 0.22365188130326458 	 0.17726044212959832
9 CTELENUM 	 0.5751291388639675 	 0.49432338851812974
10 PVTRESD1 	 0.2876133298650896 	 0.24725290256738108
11 COLGHOUS 	 9.752083745043959e-05 	 0.009874782383776894
12 STATERES 	 0.5751260913377972 	 0.49432385168307663
13 CELLFON3 	 0.44418151063651373 	 0.42592309411223267
14 LADULT 	 8.075944351768325e-05 	 0.008507186635870337
15 NUMADULT 	 0.05158090420075692 	 0.053722774656453386
16 NUMMEN 	 0.025596680228015926 	 0.03418443595340769
17 NUMWOMEN 	 0.05701586237314798 	 0.06320665381728963
18 CTELNUM1 	 0.42487086103603244 	

156 STREHAB1 	 0.0002884991441498616 	 0.0077727605325577966
157 CVDASPRN 	 0.008129106753058361 	 0.0401541039967283
158 ASPUNSAF 	 0.009526566809286627 	 0.05572597801682155
159 RLIVPAIN 	 0.002579561596400992 	 0.023711124262861295
160 RDUCHART 	 0.0023953555700817375 	 0.026785423093287946
161 RDUCSTRK 	 0.0024336189542312854 	 0.029141523497538905
162 ARTTODAY 	 0.011056424946926382 	 0.05635807173581417
163 ARTHWGT 	 0.008289609798043145 	 0.0417760418053127
164 ARTHEXER 	 0.0073750133328450524 	 0.039290162288692344
165 ARTHEDU 	 0.00935488950167632 	 0.04512883538597051
166 TETANUS 	 0.03487385781219981 	 0.12219816632278097
167 HPVADVC2 	 0.006445856464831882 	 0.04898450544076237
168 HPVADSHT 	 0.0004886200293573418 	 0.018158816950834968
169 SHINGLE2 	 0.012678047483703013 	 0.05383696713735126
170 HADMAM 	 0.006936169564279065 	 0.03270959790021226
171 HOWLONG 	 0.008662762446931817 	 0.05167082983413685
172 HADPAP2 	 0.006352737609619192 	 0.030933170071549748
173 LASTPAP2

311 _PAREC1 	 0.31107090069247145 	 0.2994198741994218
312 _PASTAE1 	 0.21720138052664034 	 0.305635309366715
313 _LMTACT1 	 0.3001667674010022 	 0.14778091047043324
314 _LMTWRK1 	 0.3109032021000158 	 0.15121071484733953
315 _LMTSCL1 	 0.40352632638895547 	 0.14285396222697216
316 _RFSEAT2 	 0.10255268410739292 	 0.2944160224642004
317 _RFSEAT3 	 0.11029873375149735 	 0.2932580652592329
318 _FLSHOT6 	 0.09053285994992892 	 0.20655093391650164
319 _PNEUMO2 	 0.09515392546661101 	 0.2239252872089939
320 _AIDTST3 	 0.19715598085257277 	 0.16468531474273926


Just found several features have skewed distribution. Thus, we applied log transform to skewed feature values.

In [359]:
def skewness(data):
    mean = np.mean(data)
    stddev = np.std(data)    
    skew = np.mean(((data - mean) / stddev) ** 3)
    return skew

def log_transform(arr, epsilon=1e-10):
    return np.log(arr + epsilon)

We calculated skewness for each feature, and if |skewness| > 1, we apply log transform for that column.

In [360]:
for i in range(scaled_x_train.shape[1]):
    transformed_col = skewness(scaled_x_train[:, i])
    if np.max(np.abs(transformed_col)) > 1:
        scaled_x_train[:, i] = log_transform(scaled_x_train[:, i])
        scaled_x_test[:,i] = log_transform(scaled_x_test[:,i])

### Split train set into validation set and train set

In [361]:
indices = np.arange(x_train.shape[0])
np.random.seed(0)
np.random.shuffle(indices)

split_idx = int(0.9 * x_train.shape[0])
y_train[y_train==-1] = 0

x_train_split = scaled_x_train[indices[:split_idx]]
y_train_split = y_train[indices[:split_idx]]

x_val = scaled_x_train[indices[split_idx:]]
y_val = y_train[indices[split_idx:]]

In [362]:
import numpy as np


def sigmoid(t):
    """apply sigmoid function on t.

    Args:
        t: scalar or numpy array

    Returns:
        scalar or numpy array
    """
    threshold = 20.0
    t = np.clip(t, -threshold, threshold)
    # Added clip to avoid overflow
    return 1.0 / (1 + np.exp(-t))


def compute_gradient_mse(y, tx, w):
    """
    Compute the gradient for mean squared error.
    """
    e = y - tx @ w
    gradient = -tx.T @ e / len(y)
    return gradient


def mean_squared_error_gd(y, tx, initial_w, max_iters, gamma):
    """
    Calculate MSE for GD
    """
    w = initial_w
    for _ in range(max_iters):
        gradient = compute_gradient_mse(y, tx, w)
        w = w - gamma * gradient
    loss = 0.5 * np.mean((y - tx @ w) ** 2)
    return w, loss


def mean_squared_error_sgd(y, tx, initial_w, max_iters, gamma):
    """
    Calculate MSE for SGD
    """
    w = initial_w
    for _ in range(max_iters):
        i = np.random.randint(0, len(y))
        gradient = compute_gradient_mse(y[i : i + 1], tx[i : i + 1], w)
        w = w - gamma * gradient
    loss = 0.5 * np.mean((y - tx @ w) ** 2)
    return w, loss


def compute_gradient_logistic(y, tx, w):
    """
    Compute the gradient for logistic regression.
    """
    prediction = sigmoid(tx @ w)
    gradient = tx.T @ (prediction - y) / len(y)
    return gradient


def logistic_regression(y, tx, initial_w, max_iters, gamma):
    """
    calculate logistic regression loss
    """
    w = initial_w
    for _ in range(max_iters):
        gradient = compute_gradient_logistic(y, tx, w)
        w = w - gamma * gradient
    loss = np.mean(y * np.log(sigmoid(tx @ w)) + (1 - y) * np.log(1 - sigmoid(tx @ w)))
    return w, -loss


def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    """
    Regulatized logistic regression loss
    """
    w = initial_w
    for _ in range(max_iters):
        gradient = compute_gradient_logistic(y, tx, w) + 2 * lambda_ * w
        w = w - gamma * gradient
    loss = np.mean(y * np.log(sigmoid(tx @ w)) + (1 - y) * np.log(1 - sigmoid(tx @ w)))
    return w, -loss


def least_squares(y, tx):
    """calculate the least squares."""
    w = np.linalg.solve(tx.T @ tx, tx.T @ y)
    e = y - tx @ w
    loss = 0.5 * np.mean(e ** 2)

    return w, loss


def ridge_regression(y, tx, lambda_):
    """implement ridge regression.

    Args:
        y: numpy array of shape (N,), N is the number of samples.
        tx: numpy array of shape (N,D), D is the number of features.
        lambda_: scalar.

    Returns:
        w: optimal weights, numpy array of shape(D,), D is the number of features.
    """
    a = tx.T @ tx + 2 * tx.shape[0] * lambda_ * np.eye(tx.shape[1])
    b = tx.T @ y
    w = np.linalg.solve(a, b)

    e = y - tx @ w
    loss = 0.5 * np.mean(e ** 2)
    return w, loss


In [ ]:
def predict_least_squares(x, w):
    return np.where(x@w >= 0, 1, 0)

def predict_ridge(x,w):
    return np.where(x@w >= 0, 1, 0)

def predict_log(x, w):
    probabilities = sigmoid(x @ w)
    predictions = np.where(probabilities > 0.5, 1, 0)
    return predictions

In [ ]:
def evaluate(y_true, y_predict):
    TP = np.sum((y_true == 1) & (y_predict == 1))
    FP = np.sum((y_true == 0) & (y_predict == 1))
    FN = np.sum((y_true == 1) & (y_predict == 0))
    TN = np.sum((y_true == 0) & (y_predict == 0))
    accuracy = (TP + TN) / (TP + FP + FN + TN)
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    recall = TP / (TP + FN) if TP + FN != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0
    return accuracy, f1_score

In [363]:
np.random.seed(0)
w_init = np.random.randn(x_train_split.shape[1])
w_init = np.clip(w_init, -1, 1)
w_init -= np.mean(w_init)

In [364]:
def weighted_compute_gradient_logistic(y, tx, w, sample_weights):
    pred = sigmoid(tx @ w)
    grad = tx.T @ (sample_weights * (pred - y))
    return grad

def reg_logistic_regression_with_val(y, tx, lambda_, initial_w, max_iters, gamma, val_y, val_tx):
    w = initial_w
    max_w = w
    max_f1 = 0
    class_weights = {0:len(y[y==1])/len(y), 1:len(y[y==0])/len(y)}
    sample_weights = class_weights[0] * (y == 0) + class_weights[1] * (y == 1)
    early_stop_count = 0
    for i in range(max_iters):
        gradient = weighted_compute_gradient_logistic(y, tx, w,sample_weights) + 2 * lambda_ * w
        w = w - gamma * gradient
        
        if i % 100 == 99:
            val_pred = predict_log(val_tx,w)
            val_acc, val_f1 = evaluate(val_y,val_pred)
            if val_f1 > max_f1:
                max_f1 = val_f1
                max_w = w
                early_stop_count = 0
            else:
                early_stop_count += 1
                if early_stop_count > 7:
                    print('early_stop')
                    break
            print(f"Iteration {i + 1}/{max_iters}:",val_acc,val_f1)
        
    loss = np.mean(y * np.log(sigmoid(tx @ max_w)) + (1 - y) * np.log(1 - sigmoid(tx @ max_w)))
    return max_w, -loss

In [356]:
w, loss = reg_logistic_regression_with_val(y_train_split, x_train_split, 0.1,w_init, 5000, 1e-4, y_val, x_val)

Iteration 50/5000: 0.39068690193210215 0.21947220487195498
Iteration 100/5000: 0.4072347168891327 0.22434900506440164
Iteration 150/5000: 0.416529530078625 0.22754780924715565
Iteration 200/5000: 0.42332540988602424 0.22961364654154623
Iteration 250/5000: 0.4280490034741269 0.23119777158774377
Iteration 300/5000: 0.43130980678978487 0.23215240916759247
Iteration 350/5000: 0.4332297190223685 0.23275577557755775
Iteration 400/5000: 0.43636862314865604 0.2337490160334756
Iteration 450/5000: 0.4385628085573231 0.2343847400573494
Iteration 500/5000: 0.44066556957396236 0.23505876469117282
Iteration 550/5000: 0.44307307856402756 0.2356434815341503
Iteration 600/5000: 0.44462729322850003 0.2362112321877619
Iteration 650/5000: 0.4463034070823429 0.23682110303692191
Iteration 700/5000: 0.4480099957335284 0.2373152553791739
Iteration 750/5000: 0.4488632900591211 0.23753109321640875
Iteration 800/5000: 0.450082281952825 0.23799670622017652
Iteration 850/5000: 0.45102700067044554 0.238373076272619

In [ ]:
best_weight = w_init
best_f1 = 0
weight_save_dict = dict()
for lambda_ in [0,0.2,0.4,0.6,0.8,1]:
    for lr in [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]:
        print(lambda_, lr)
        w, loss = reg_logistic_regression_with_val(
            y_train_split, x_train_split, lambda_ ,w_init, 10000, lr, y_val, x_val
        )
        weight_save_dict[(lambda_, lr)] = w
        y_val_pred = predict_log(x_val,w)
        acc, f1 = evaluate(y_val,y_val_pred)
        if best_f1 < f1:
            best_f1 = f1
            best_weight = w

0 1e-05
Iteration 100/10000: 0.8861461571280551 0.27399922269724053
Iteration 200/10000: 0.8861766319253978 0.31024930747922436
Iteration 300/10000: 0.8875784726031571 0.3205010130779149
Iteration 400/10000: 0.8881879685500091 0.33084078059456495
Iteration 500/10000: 0.8885841409154629 0.3407861521817527
Iteration 600/10000: 0.8899555067958798 0.3461886655803006
Iteration 700/10000: 0.8906259523374169 0.35228298141129755
Iteration 800/10000: 0.8911135490948985 0.35400470077743634
Iteration 900/10000: 0.8911135490948985 0.35586803677663603
Iteration 1000/10000: 0.8909002255135003 0.35541951746489014
Iteration 1100/10000: 0.8909611751081855 0.3567062207838907
Iteration 1200/10000: 0.8912049734869263 0.3569884726224784
Iteration 1300/10000: 0.8915401962576949 0.3593159315931593
Iteration 1400/10000: 0.8918144694337783 0.3610511159107272
Iteration 1500/10000: 0.8916925702444078 0.3619389587073608
Iteration 1600/10000: 0.8919058938258061 0.36376681614349776
Iteration 1700/10000: 0.891936368

Iteration 2100/10000: 0.89160114585238 0.3644809719492585
Iteration 2200/10000: 0.8914182970683245 0.3652235880990558
Iteration 2300/10000: 0.8916925702444078 0.364903502501787
Iteration 2400/10000: 0.8917230450417505 0.3654223968565815
Iteration 2500/10000: 0.89160114585238 0.3653880463871544
Iteration 2600/10000: 0.8916620954470653 0.3655184722470105
Iteration 2700/10000: 0.8917535198390931 0.3663931501962184
Iteration 2800/10000: 0.8918144694337783 0.3665239114917916
Iteration 2900/10000: 0.8917839946364357 0.366232375513118
Iteration 3000/10000: 0.8919058938258061 0.3667202285306196
Iteration 3100/10000: 0.8919363686231486 0.3674634320371031
Iteration 3200/10000: 0.8919058938258061 0.3671721677074041
Iteration 3300/10000: 0.8918754190284635 0.3668807994289793
Iteration 3400/10000: 0.8919363686231486 0.3667857142857144
Iteration 3500/10000: 0.8919668434204913 0.36730323041227914
Iteration 3600/10000: 0.8919973182178339 0.3680456490727532
Iteration 3700/10000: 0.8919668434204913 0.36

Iteration 900/10000: 0.8914792466630097 0.3552417164584465
Iteration 1000/10000: 0.8912659230816115 0.35595667870036096
Iteration 1100/10000: 0.891083074297556 0.356036036036036
Iteration 1200/10000: 0.8913878222709819 0.357606344628695
Iteration 1300/10000: 0.8916925702444078 0.35917778579156145
Iteration 1400/10000: 0.8916925702444078 0.35963963963963963
Iteration 1500/10000: 0.8917535198390931 0.3606911447084233
Iteration 1600/10000: 0.8919363686231486 0.36291771469637085
Iteration 1700/10000: 0.8920582678125191 0.3634076204169663
Iteration 1800/10000: 0.892210641799232 0.36601541494891554
Iteration 1900/10000: 0.8920277930151764 0.36607622114868493
Iteration 2000/10000: 0.8919973182178339 0.3660107334525939
Iteration 2100/10000: 0.8918144694337783 0.36561829878484636
Iteration 2200/10000: 0.8916316206497227 0.3645461043602573
Iteration 2300/10000: 0.8917839946364357 0.3650992311818344
Iteration 2400/10000: 0.8917839946364357 0.3650992311818344
Iteration 2500/10000: 0.89175351983909

Iteration 3000/10000: 0.8937039068690193 0.3523950984032677
Iteration 3100/10000: 0.8937039068690193 0.35167286245353163
Iteration 3200/10000: 0.8937039068690193 0.3499813641446142
Iteration 3300/10000: 0.8937953312610472 0.3501771396606377
Iteration 3400/10000: 0.8937648564637045 0.35180364447750095
Iteration 3500/10000: 0.8937343816663619 0.35294117647058815
Iteration 3600/10000: 0.8935515328823064 0.35302833858121874
early_stop
0.4 1e-05
Iteration 100/10000: 0.8861461571280551 0.27399922269724053
Iteration 200/10000: 0.8862071067227403 0.310306612486147
Iteration 300/10000: 0.8878222709818979 0.3209739900387383
Iteration 400/10000: 0.8881574937526665 0.3307804522246536
Iteration 500/10000: 0.8885841409154629 0.34054834054834054
Iteration 600/10000: 0.8899250319985372 0.34588917059036584
Iteration 700/10000: 0.8906564271347596 0.3523465703971119
Iteration 800/10000: 0.891083074297556 0.35370705244122963
Iteration 900/10000: 0.8911744986895838 0.35506592017337907
Iteration 1000/10000:

In [329]:
w, loss = reg_logistic_regression_with_val(y_train_split, x_train_split, 0.1,w, 5000, 1e-4, y_val, x_val)

Iteration 50/5000: 0.7566587432193576 0.35817056506711675
Iteration 100/5000: 0.8677393795331261 0.3921568627450981
Iteration 150/5000: 0.8882489181446943 0.37412527735108375
Iteration 200/5000: 0.8880355945632962 0.37239494362828834
Iteration 250/5000: 0.8890717376729445 0.3728463128876637
Iteration 300/5000: 0.8895898092277686 0.37133437445774775
Iteration 350/5000: 0.8901078807825928 0.3719958202716824
Iteration 400/5000: 0.890473578350704 0.37123862841147653
Iteration 450/5000: 0.8906869019321022 0.37169381678052194
early_stop


In [330]:
y_val_predicted = predict_log(x_val,w)

In [332]:
evaluate(y_val, y_val_predicted)

(0.8677393795331261, 0.3921568627450981)

In [ ]:
y_test_predicted = predict_log(scaled_x_test, best_weight)

In [ ]:
y_test_predicted[y_test_predicted==0] = -1

In [321]:
def create_csv_submission(ids, y_pred, name):
    """
    This function creates a csv file named 'name' in the format required for a submission in Kaggle or AIcrowd.
    The file will contain two columns the first with 'ids' and the second with 'y_pred'.
    y_pred must be a list or np.array of 1 and -1 otherwise the function will raise a ValueError.

    Args:
        ids (list,np.array): indices
        y_pred (list,np.array): predictions on data correspondent to indices
        name (str): name of the file to be created
    """
    # Check that y_pred only contains -1 and 1
    if not all(i in [-1, 1] for i in y_pred):
        raise ValueError("y_pred can only contain values -1, 1")

    with open(name, "w", newline="") as csvfile:
        fieldnames = ["Id", "Prediction"]
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({"Id": int(r1), "Prediction": int(r2)})


In [ ]:
create_csv_submission(test_ids,y_test_predicted,'best.csv')